# <b><u>Connection to DB</u></b>

This is the connection link to my database on postgreSQL, the actual connection function is on the file **db_connect.py**

In [ ]:
# Import necessary packages
import pandas as pd
from db_connect import connect_to_db

# Step 1: Connect to the database
conn = connect_to_db()

# Step 2: Create a cursor and run a query
cursor = conn.cursor()YlGnBu
query = "SELECT * FROM food_prices_cleaned.food_prices_kenya;"
cursor.execute(query)

# Step 3: Fetch results and convert to a DataFrame
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Step 4: Display the data
print("Connection successful! Previewing data:")
display(df.head(25))


Data Exploration with Python, tryna get to understand my data

In [ ]:
df.info()

In [ ]:
df.describe()

---

# <b><u>Data Manipulation and Modification</u></b>

## Standardising potatoes data to price per KG

Divide potatoes price columns by 50 to standardize to price per 1kg
I choose to do this on derived columns to avoid confusion, or incase i'll need the original data in future.

In [ ]:
df['o_potatoes_1kg'] = df['o_potatoes'] / 50
df['h_potatoes_1kg'] = df['h_potatoes'] / 50
df['l_potatoes_1kg'] = df['l_potatoes'] / 50
df['c_potatoes_1kg'] = df['c_potatoes'] / 50

## confirm if the additional potatoes columns have been added


In [ ]:
df.info()

Renaming some columns for better understanding of what they represent

In [ ]:
import pandas as pd

def rename_agric_columns(df):
    """
    Renames columns like o_beans, h_beans, c_maize, etc. 
    to a consistent format such as beans_open, maize_high, etc.
    """
    rename_map = {}
    prefix_map = {
        'o': 'open',
        'h': 'high',
        'l': 'low',
        'c': 'close'
    }

    # Iterate through existing columns
    for col in df.columns:
        # Check for trading-style prefixes (o_, h_, l_, c_)
        for prefix, new_prefix in prefix_map.items():
            if col.startswith(f"{prefix}_"):
                # Example: o_beans → beans_open
                rename_map[col] = f"{col.split('_', 1)[1]}_{new_prefix}"
                break
        # Handle inflation_* and trust_* as is
        if col.startswith("inflation_") or col.startswith("trust_"):
            rename_map[col] = col  # keep same (optional)
    
    # Apply renaming
    df = df.rename(columns=rename_map)
    return df

In [ ]:
df = rename_agric_columns(df)

In [ ]:
df.columns

In this step i'm grouping the data by province, then ordering it by year: This keeps all rows but arranges them so that:
All rows from the same province are grouped together
Within each province, data appears in chronological order

In [ ]:
df = df.sort_values(['provinces', 'year']).reset_index(drop=True)
df

---

# <b><u> VISUALS </u></b>

## Visualizing trends by Time(Yearly and Monthly)

## General Trends and Overview

What are the overall trends in food prices **(beans, maize, potatoes, and the food price index)** across Kenya over the years (2007–2025)?

We want to see how prices have changed over time, for each commodity across all regions.
That means we’ll probably focus on averages per year (national trend), not per province yet.

We'll start by calculating the mean closing price for each commodity per year:

# Yearly Trends for Food 

In [ ]:
yearly_trends = df.groupby('year')[['beans_close','maize_close','potatoes_1kg_close','food_price_index_close']].mean().reset_index()
yearly_trends

## Overall Food Price Trends in Kenya by Year (2007–2025):

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.lineplot(data=yearly_trends, x='year', y='beans_close', label='Beans')
sns.lineplot(data=yearly_trends, x='year', y='maize_close', label='Maize')
sns.lineplot(data=yearly_trends, x='year', y='potatoes_1kg_close', label='Potatoes')
sns.lineplot(data=yearly_trends, x='year', y='food_price_index_close', label='Food_Price_Index')

plt.title('Overall Food Price Trends in Kenya (2007–2025)', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Average Closing Price')
plt.legend(title='Commodity')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

## Breakdown based on the Line chart above 

### 1. Upward Trend Overall:
All commodities **beans**, **maize**, and **potatoes** show a gradual increase in average prices from 2007 through around 2023, followed by a slight decline toward 2025.  
This suggests **long-term inflationary pressure on food prices in Kenya**.
### 2. Beans are consistently the most expensive:
The **blue line(Beans)** remains well above maize and potatoes throughout the period.
This likely reflects both higher production costs and strong demand for beans as a protein source
### 3. Parallel movement between maize and potatoes
The **Orange(maize)** and **green(Potatoes)** line move roughly together, meaning price changes for one often coincide with the other
This may reflect **shared market influence** like weather condition or fuel prices that affect all stable crops.
### 4. Food Price Index follows the same direction:
Even though its values are smaller in scale, the **Food Price Index (red line)** mirrors the general direction of the other commodities.
It acts as a **summary indicator** of overall food inflation, showing peaks and troughs that align with the crops’ price changes.
### 5. Notable peaks (2022–2023):
There’s a sharp spike across all commodities around 2022–2023, likely due to **global and local disruptions** e.g., drought, COVID-19 aftereffects, global supply chain issues or elections.
After this spike, prices dip slightly toward 2025, suggesting a partial recovery or stabilization.
### 6. Notable Dips
There are **two visible dips**, around **2010** and **2018**, across most commodities.  
These years coincide with **major election periods in Kenya** (the 2010 constitutional referendum and the 2017 general election).  
Such events often influence food prices through **market disruptions, political uncertainty, and short-term policy changes** that affect production and distribution.


# Monthly Trends for Food

In [ ]:
monthly_trends = df.groupby(['year','month'])[['beans_close','maize_close','potatoes_1kg_close','food_price_index_close']].mean().reset_index()
monthly_trends

# Visuals For Monthly Trends

In [ ]:
import calendar

def month_to_name(x):
    # If it's an integer (1–12), convert to month name
    if isinstance(x, int):
        return calendar.month_name[x]
    # If it's already a string (e.g. "January"), keep it as is
    return x

monthly_trends['month'] = monthly_trends['month'].apply(month_to_name)

# Heat map for Beans

In [ ]:
# Make sure month names are properly formatted and ordered
month_order = list(calendar.month_name[1:])

# Convert month column to an ordered categorical type
monthly_trends['month'] = pd.Categorical(monthly_trends['month'],
                                         categories=month_order,
                                         ordered=True)

# pivot will respect that order
pivot = monthly_trends.pivot(index='year', columns='month', values='beans_close')

plt.figure(figsize=(10,6))
sns.heatmap(pivot, cmap='RdYlGn_r', annot=True, fmt=".1f")
plt.title('Seasonal Heatmap of Beans (Monthly Averages)')
plt.xlabel('Month')
plt.ylabel('Year')
plt.show()

# Heat map for Maize

In [ ]:
# Make sure month names are properly formatted and ordered
month_order = list(calendar.month_name[1:])

# Convert month column to an ordered categorical type
monthly_trends['month'] = pd.Categorical(monthly_trends['month'],
                                         categories=month_order,
                                         ordered=True)

# pivot will respect that order
pivot = monthly_trends.pivot(index='year', columns='month', values='maize_close')

plt.figure(figsize=(10,6))
sns.heatmap(pivot, cmap='RdYlGn_r', annot=True, fmt=".1f")
plt.title('Seasonal Heatmap of Maize (Monthly Averages)')
plt.xlabel('Month')
plt.ylabel('Year')
plt.show()

# Heat map for potatoes

In [ ]:
# Make sure month names are properly formatted and ordered
month_order = list(calendar.month_name[1:])

# Convert month column to an ordered categorical type
monthly_trends['month'] = pd.Categorical(monthly_trends['month'],
                                         categories=month_order,
                                         ordered=True)

# pivot will respect that order
pivot = monthly_trends.pivot(index='year', columns='month', values='potatoes_1kg_close')

plt.figure(figsize=(10,6))
sns.heatmap(pivot, cmap='RdYlGn_r', annot=True, fmt=".1f")
plt.title('Seasonal Heatmap of Potatoes (Monthly Averages)')
plt.xlabel('Month')
plt.ylabel('Year')
plt.show()

# Heat map for Food Prices Index

In [ ]:
import calendar
# Make sure month names are properly formatted and ordered
month_order = list(calendar.month_name[1:])

# Convert month column to an ordered categorical type
monthly_trends['month'] = pd.Categorical(monthly_trends['month'],
                                         categories=month_order,
                                         ordered=True)

# pivot will respect that order
pivot = monthly_trends.pivot(index='year', columns='month', values='food_price_index_close')

plt.figure(figsize=(10,6))
sns.heatmap(pivot, cmap='RdYlGn_r', annot=True, fmt=".1f")
plt.title('Seasonal Heatmap of Food Price Index (Monthly Averages)')
plt.xlabel('Month')
plt.ylabel('Year')
plt.show()

# Interpretation of the Heatmap: Seasonal Food Price Index (2007–2025)
#### 1.**General Trend Over Time (2007-2025)**
The **color intensity deepens gradually from top to bottom**, indicating that **food prices have generally increased** over the years.
Earlier years (2007–2010) show light yellow-green colors (index ≈ 0.4–0.8), while recent years (2023–2025) show dark blue shades (index ≈ 1.3–1.7).
**Conclusion:** There’s a clear long-term upward trend in food prices, reflecting inflation, cost of production, or other macroeconomic factors.

#### 2.**Seasonal (Month-to-Month) Variation**

Prices tend to **peak** between **May and August (darker shades)**.
**January to April** often shows **lighter colors**, relatively lower food prices.
September to December stabilizes or slightly cools off, depending on the year.
**Interpretation:**
This suggests **seasonal price pressure mid-year**, possibly linked to:
**Planting or lean** seasons before harvest (reduced supply).
**Increased demand** or transport challenges in those months.
Conversely, **lower prices early in the year** may correspond to harvest periods when supply is abundant.

#### 3. **Exceptional Years**

**2023** shows the **darkest overall colors** (1.6–1.8 range), indicating **exceptionally high prices** likely due to **inflationary pressures or external shocks** (e.g., global food shortages, climate effects).
2010 and 2016 show relatively cooler colors even mid-year, meaning prices were comparatively stable during those periods.
**Conclusion:** Prices have become consistently high but less seasonally volatile in recent years.



---

## Visualizing trends by Regions(Province, County and Market)

## Regions that consistently have the highest or lowest food prices By Province

#### Average Closing Price of Food Per Province

In [ ]:
# Step 1: Group by provinces and year to get average closing prices per year
avg_prices_province = (
    df.groupby(['provinces','year'], as_index=False)[['beans_close', 'maize_close','potatoes_1kg_close','food_price_index_close']].mean()
)
avg_prices_province

#### Visualize the table above (Average Closing Price of Food Per Province)

In [ ]:
# Step 1: Reshape your data from wide to long format
price_columns = ['beans_close', 'maize_close', 'potatoes_1kg_close', 'food_price_index_close']
df_long = avg_prices_province.melt(
    id_vars=['provinces', 'year'],
    value_vars=price_columns,
    var_name='Commodity',
    value_name='Price'
)

print("--- Head of Reshaped (Long) Data ---")
print(df_long.head())
print("\n")

# 2. Plot the data using sns.relplot
sns.set_style("whitegrid")

# This creates a Figure-level plot that automatically handles faceting
g = sns.relplot(
    data=df_long,
    kind='line',          # Specify a line plot
    x='year',
    y='Price',
    hue='Commodity',      # Color lines by commodity
    col='provinces',      # Create columns for each province
    col_wrap=3,           # Wrap the columns after 3 plots
    height=3.5,           # Height of each individual plot
    aspect=1.5,           # Aspect ratio (width/height)
    legend='full'
)

# Step 3: Customize titles and labels
g.set_axis_labels('Year', 'Average Closing Price')
g.set_titles(col_template='{col_name}')
g.fig.suptitle('Food Price Trends by Province (2007–2025)', fontsize=16, y=1.03) # Add a main title

# 4. Save the figure
#plt.tight_layout()
plt.savefig('province_price_trends.png')

print("Faceted plot saved to 'province_price_trends.png'")

### Province with consistently the Highest and Lowest Food Price

In [ ]:
# Created a New Column average_food_price
avg_prices_province['avg_food_price'] = avg_prices_province[
   ['beans_close', 'maize_close', 'potatoes_1kg_close']
].mean(axis=1)

#### Highest & Lowest Overall Average Food Prices By Province

In [ ]:
# Compute the long-term average for each province
province_avg_overall = (
    avg_prices_province.groupby('provinces', as_index=False)['avg_food_price']
    .mean()
    .sort_values(by='avg_food_price', ascending=False)
)

# Display results
print(province_avg_overall)

# Get the top province
top_province = province_avg_overall.iloc[0]
bottom_province = province_avg_overall.iloc[-1]
print(f"Province with consistently highest food price: {top_province['provinces']} ({top_province['avg_food_price']:.2f})")
print(f"Province with consistently lowest food price: {bottom_province['provinces']} ({bottom_province['avg_food_price']:.2f})")


#### Higest & Lowest Overall Prices by Province for each crop

In [ ]:
# List of commodities to analyze
crops = ['beans_close', 'maize_close', 'potatoes_1kg_close', 'food_price_index_close']

for crop in crops:
    crop_name = crop.replace('_close', '').capitalize()
    print(f"\n🌾 Ranking of provinces by average {crop_name} price:\n")

    # Compute average price per province
    province_avg = (
        avg_prices_province.groupby('provinces', as_index=False)[crop]
        .mean()
        .sort_values(by=crop, ascending=False)
    )

    # Print full ranking
    for i, row in province_avg.iterrows():
        print(f"{i+1}. {row['provinces']}: {row[crop]:.2f}")

    # Identify top and bottom provinces
    top = province_avg.iloc[0]
    bottom = province_avg.iloc[-1]

    # Print summary
    print(f"\n🏆 Highest average {crop_name} price: {top['provinces']} ({top[crop]:.2f})")
    print(f"💰 Lowest average {crop_name} price: {bottom['provinces']} ({bottom[crop]:.2f})")
    print("-" * 60)

----

## Regions that consistently have the highest or lowest food prices By County

#### Average Closing price of Food per County

In [ ]:
# Step 2: Group by counties and year to get average closing prices per year
avg_prices_county = (
    df.groupby(['counties','year'], as_index=False)[['beans_close', 'maize_close','potatoes_1kg_close','food_price_index_close']].mean()
)
avg_prices_county

#### Visualize the table above (Average Closing Price of Food Per County)

In [ ]:
# Step 1: Keep province info for later merge
county_province_map = df[['counties', 'provinces']].drop_duplicates()

# Step 2: Group by counties and year (correct averaging)
avg_prices_county = (
    df.groupby(['counties', 'year'], as_index=False)[
        ['beans_close', 'maize_close', 'potatoes_1kg_close', 'food_price_index_close']
    ].mean()
)

# Step 3: Merge province info back
avg_prices_county = avg_prices_county.merge(county_province_map, on='counties', how='left')

# Step 4: Append province name to county
avg_prices_county['county_label'] = avg_prices_county['counties'] + " (" + avg_prices_county['provinces'] + ")"

# Step 5: Reshape for visualization
price_columns = ['beans_close', 'maize_close', 'potatoes_1kg_close', 'food_price_index_close']
df_long = avg_prices_county.melt(
    id_vars=['provinces', 'county_label', 'year'],
    value_vars=price_columns,
    var_name='Commodity',
    value_name='Price'
)

# Step 6: Plot
sns.set_style("whitegrid")

g = sns.relplot(
    data=df_long,
    kind='line',
    x='year',
    y='Price',
    hue='Commodity',
    col='county_label',    # use new county label
    col_wrap=3,
    height=3.5,
    aspect=1.5,
    legend='full'
)

g.set_axis_labels('Year', 'Average Closing Price')
g.set_titles(col_template='{col_name}')
g.fig.suptitle('Food Price Trends by County (2007–2025)', fontsize=16, y=1.03)

plt.savefig('county_price_trends_labeled.png', bbox_inches='tight')
print("Faceted plot saved to 'county_price_trends_labeled.png'")


### Counties with consistently the Highest & Lowest Food Price

In [ ]:
# Created a New Column average_food_price
avg_prices_county['avg_food_price'] = avg_prices_county[
    ['beans_close', 'maize_close', 'potatoes_1kg_close']
].mean(axis=1)
avg_prices_county

#### Highest & Lowest Overall Average Food Prices By county

In [ ]:
# Compute the long-term average for each County
county_avg_overall = (
    avg_prices_county.groupby('counties', as_index=False)['avg_food_price']
    .mean()
    .sort_values(by='avg_food_price', ascending=False)
)

# Display results
print(county_avg_overall)

# Get the top & bottom County
top_county = county_avg_overall.iloc[0]
bottom_county = county_avg_overall.iloc[-1]
print(f"County with consistently highest food price: {top_county['counties']} ({top_county['avg_food_price']:.2f})")
print(f"County with consistently lowest food price: {bottom_county['counties']} ({bottom_county['avg_food_price']:.2f})")


#### Higest & Lowest Overall Prices by county for each crop

In [ ]:
# List of commodities to analyze
crops = ['beans_close', 'maize_close', 'potatoes_1kg_close', 'food_price_index_close']

for crop in crops:
    crop_name = crop.replace('_close', '').capitalize()
    print(f"\n🌾 Ranking of counties by average {crop_name} price:\n")

    # Compute average price per county
    county_avg = (
        avg_prices_county.groupby('counties', as_index=False)[crop]
        .mean()
        .sort_values(by=crop, ascending=False)
    )

    # Print full ranking
    for i, row in county_avg.iterrows():
        print(f"{i+1}. {row['counties']}: {row[crop]:.2f}")

    # Identify top and bottom county
    top = county_avg.iloc[0]
    bottom = county_avg.iloc[-1]

    # Print summary
    print(f"\n🏆 Highest average {crop_name} price: {top['counties']} ({top[crop]:.2f})")
    print(f"💰 Lowest average {crop_name} price: {bottom['counties']} ({bottom[crop]:.2f})")
    print("-" * 60)

----

## Regions that consistently have the highest or lowest food prices By Market

### Average Closing Price for Food per Market

In [ ]:
# Step 3: Group by market and year to get average closing prices per year
avg_prices_mkt = (
    df.groupby(['mkt_name','year'], as_index=False)[['beans_close', 'maize_close','potatoes_1kg_close','food_price_index_close']].mean()
)
avg_prices_mkt

#### Visualize the table above (Average Closing Price of Food Per Market)

In [ ]:
# Step 1: Create a mapping of market → county & province
market_location_map = df[['mkt_name', 'counties', 'provinces']].drop_duplicates()

# Step 2: Group by market and year to compute averages (if not already done)
avg_prices_mkt = (
    df.groupby(['mkt_name', 'year'], as_index=False)[
        ['beans_close', 'maize_close', 'potatoes_1kg_close', 'food_price_index_close']
    ].mean()
)

# Step 3: Merge location info into the averages
avg_prices_mkt = avg_prices_mkt.merge(market_location_map, on='mkt_name', how='left')

# Step 4: Append county and province to market name
avg_prices_mkt['mkt_label'] = (
    avg_prices_mkt['mkt_name']
    + " (" 
    + avg_prices_mkt['counties'] 
    + ", " 
    + avg_prices_mkt['provinces'] 
    + ")"
)

# Step 5: Reshape data for plotting
price_columns = ['beans_close', 'maize_close', 'potatoes_1kg_close', 'food_price_index_close']
df_long = avg_prices_mkt.melt(
    id_vars=['mkt_label', 'year'],
    value_vars=price_columns,
    var_name='Commodity',
    value_name='Price'
)

print("--- Head of Reshaped (Long) Data ---")
print(df_long.head(), "\n")

# Step 6: Plot
sns.set_style("whitegrid")

g = sns.relplot(
    data=df_long,
    kind='line',
    x='year',
    y='Price',
    hue='Commodity',
    col='mkt_label',     # use new combined label
    col_wrap=3,
    height=3.5,
    aspect=1.5,
    legend='full'
)

g.set_axis_labels('Year', 'Average Closing Price')
g.set_titles(col_template='{col_name}')
g.fig.suptitle('Food Price Trends by Market (2007–2025)', fontsize=16, y=1.03)

plt.savefig('Market_price_trends_labeled.png', bbox_inches='tight')
print("Faceted plot saved to 'Market_price_trends_labeled.png'")


### Markets with consistently the Highest & Lowest Food Price

In [ ]:
# Created a New Column average_food_price per market
avg_prices_mkt['avg_food_price'] = avg_prices_mkt[
    ['beans_close', 'maize_close', 'potatoes_1kg_close']
].mean(axis=1)
avg_prices_mkt

#### Highest & Lowest Overall Average Food Prices By Market

In [ ]:
# Compute the long-term average for each market
mkt_avg_overall = (
    avg_prices_mkt.groupby('mkt_name', as_index=False)['avg_food_price']
    .mean()
    .sort_values(by='avg_food_price', ascending=False)
)

# Display results
print(mkt_avg_overall)

# Get the top & bottom County
top_mkt = mkt_avg_overall.iloc[0]
bottom_mkt = mkt_avg_overall.iloc[-1]
print(f"Market with consistently highest food price: {top_mkt['mkt_name']} ({top_mkt['avg_food_price']:.2f})")
print(f"Market with consistently lowest food price: {bottom_mkt['mkt_name']} ({bottom_mkt['avg_food_price']:.2f})")


#### Highest & Lowest Overall Prices by Market for each crop

In [ ]:
# List of commodities to analyze
crops = ['beans_close', 'maize_close', 'potatoes_1kg_close', 'food_price_index_close']

for crop in crops:
    crop_name = crop.replace('_close', '').capitalize()
    print(f"\n🌾 Ranking of markets by average {crop_name} price:\n")

    # Compute average price per Market
    mkt_avg = (
        avg_prices_mkt.groupby('mkt_name', as_index=False)[crop]
        .mean()
        .sort_values(by=crop, ascending=False)
    )

    # Print full ranking
    for i, row in mkt_avg.iterrows():
        print(f"{i+1}. {row['mkt_name']}: {row[crop]:.2f}")

    # Identify top and bottom Market
    top = mkt_avg.iloc[0]
    bottom = mkt_avg.iloc[-1]

    # Print summary
    print(f"\n🏆 Highest average {crop_name} price: {top['mkt_name']} ({top[crop]:.2f})")
    print(f"💰 Lowest average {crop_name} price: {bottom['mkt_name']} ({bottom[crop]:.2f})")
    print("-" * 60)

# Insights From the Charts

### 1. Universal Price Shock (2022-2023)
The most striking feature across **every single province and county is a dramatic and sharp increase in food prices** starting around 2022 and peaking in 2023.

**Nationwide Event:** The fact that this spike occurs simultaneously in all regions (from Wajir in the North Eastern to Mombasa on the Coast, and Nakuru in the Rift Valley) strongly suggests a nationwide event, such as a severe drought, major economic factors (e.g., inflation, fuel prices), or global supply chain issues affecting the entire country.

**Gradual Rise Pre-2020:** Before this spike, the general trend from 2007 to 2020 was a slow, gradual increase in prices for all commodities.

**Post-Peak Leveling:** The data from 2023 towards 2025 indicates that prices have begun to fall slightly or level off from their extreme 2023 peaks, though they remain significantly higher than pre-2022 levels.

### 2. Commodity-Specific Price Tiers
The commodities exist in distinct price brackets:

**Beans (beans_close):** This is consistently the most expensive commodity by a significant margin in all regions. It also exhibits the highest volatility, with the most dramatic peaks and troughs, and it experienced the most extreme price surge during the 2022-2023 event.

**Maize (maize_close) and Potatoes (potatoes_1kg_close):** These two commodities are much cheaper than beans. Their prices track each other very closely across the entire 18-year period and in all regions. This high correlation suggests their prices are driven by similar market forces, and they may act as substitutes for one another.

**Food Price Index (food_price_index_close):** This red line remains consistently flat and low (around a value of 10-15) in every chart. This indicates it is not a direct price but likely a composite index measured against a base year, and it is not as volatile as the prices of these individual staple foods.

### 3. Regional Variations in Price Magnitude
While all regions follow the same pattern, the magnitude of the price spikes varies.

**Highest Price Spikes:** The provincial chart shows that the North Eastern, Coast, and Eastern provinces experienced the most severe price peaks for beans (beans_close), with average prices surging well above 150.

**Supporting County Data:** This is confirmed by the county-level charts. For example, Wajir (North Eastern) and Mombasa (Coast) show some of the most extreme peaks in bean prices.

**Major Production Areas:** Regions like the Rift Valley (e.g., Nakuru, Uasin Gishu counties) also saw severe price hikes, but the peak for beans appears to be slightly less extreme than in the North Eastern or Coast regions. This could imply that regions further from production centers or with logistical challenges (like North Eastern) face greater price volatility.

# Inflation

So for this phase, we will check out the inflation across the years for each food price, by province that is, which province has the most volatile inflation prices, then we'll double up on county and market. For the visuals, maybe a bar graph

### Visualizing Inflation by Time (Yearly & Monthly)

In [ ]:
yearly_trends = df.groupby('year')[['inflation_beans','inflation_maize','inflation_potatoes','inflation_food_price_index']].mean().reset_index()
yearly_trends

## Inflation Trends over the years (2008 - 2025)

In [ ]:
# Filter out 2007 and enhance the visualization
sns.set_theme(style='whitegrid')

plt.figure(figsize=(12, 6))

# Filter data to remove 2007 and get non-NaN values
filtered_data = yearly_trends[
    (yearly_trends['year'] > 2007) & 
    (yearly_trends['inflation_food_price_index'].notna())
]

# Create the bar plot
bars = sns.barplot(
    data=filtered_data,
    x='year',
    y='inflation_food_price_index',
    color='#3498db',       # Blue for positive values
    edgecolor='#2980b9'    # Darker blue for edges
)

# Color negative bars in red
for i, bar in enumerate(bars.patches):
    if bar.get_height() < 0:
        bar.set_color('#e74c3c')    # Red for negative values
        bar.set_edgecolor('#c0392b') # Darker red for edges

# Enhance the visual style
plt.title('Food Price Inflation in Kenya (2008–2025)', 
          fontsize=16, 
          pad=20,
          weight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Inflation', fontsize=12)

# Customize grid
plt.grid(True, linestyle='--', alpha=0.3, axis='y')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add value labels on top/bottom of each bar
for i, row in enumerate(filtered_data.itertuples()):
    if pd.notna(row.inflation_food_price_index):
        value = row.inflation_food_price_index
        va = 'bottom' if value >= 0 else 'top'  # Adjust label position based on value
        plt.text(i, value, 
                f'{value:.1f}', 
                ha='center', 
                va=va,
                fontsize=10)

# Adjust layout
plt.tight_layout()

# Set y-axis limits with padding for both positive and negative values
valid_values = filtered_data['inflation_food_price_index'].dropna()
if not valid_values.empty:
    max_val = max(valid_values)
    min_val = min(valid_values)
    plt.ylim(min_val * 1.1, max_val * 1.1)  # Add 10% padding on both ends

# Add horizontal line at y=0
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)

plt.show()

## Beans Inflation over the years (2008 - 2025)

In [ ]:
# Filter out 2007 and enhance the visualization
sns.set_theme(style='whitegrid')

plt.figure(figsize=(12, 6))

# Filter data to remove 2007 and get non-NaN values
filtered_data = yearly_trends[
    (yearly_trends['year'] > 2007) & 
    (yearly_trends['inflation_beans'].notna())
]

# Create the bar plot
bars = sns.barplot(
    data=filtered_data,
    x='year',
    y='inflation_beans',
    color='#3498db',       # Blue for positive values
    edgecolor='#2980b9'    # Darker blue for edges
)

# Color negative bars in red
for i, bar in enumerate(bars.patches):
    if bar.get_height() < 0:
        bar.set_color('#e74c3c')    # Red for negative values
        bar.set_edgecolor('#c0392b') # Darker red for edges

# Enhance the visual style
plt.title('Beans Inflation in Kenya (2008–2025)', 
          fontsize=16, 
          pad=20,
          weight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Inflation', fontsize=12)

# Customize grid
plt.grid(True, linestyle='--', alpha=0.3, axis='y')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add value labels on top/bottom of each bar
for i, row in enumerate(filtered_data.itertuples()):
    if pd.notna(row.inflation_beans):
        value = row.inflation_beans
        va = 'bottom' if value >= 0 else 'top'  # Adjust label position based on value
        plt.text(i, value, 
                f'{value:.1f}', 
                ha='center', 
                va=va,
                fontsize=10)

# Adjust layout
plt.tight_layout()

# Set y-axis limits with padding for both positive and negative values
valid_values = filtered_data['inflation_beans'].dropna()
if not valid_values.empty:
    max_val = max(valid_values)
    min_val = min(valid_values)
    plt.ylim(min_val * 1.1, max_val * 1.1)  # Add 10% padding on both ends

# Add horizontal line at y=0
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)

plt.show()

## Maize Inflation over the years (2008 - 2025)

In [ ]:
# Filter out 2007 and enhance the visualization
sns.set_theme(style='whitegrid')

plt.figure(figsize=(12, 6))

# Filter data to remove 2007 and get non-NaN values
filtered_data = yearly_trends[
    (yearly_trends['year'] > 2007) & 
    (yearly_trends['inflation_maize'].notna())
]

# Create the bar plot
bars = sns.barplot(
    data=filtered_data,
    x='year',
    y='inflation_maize',
    color='#3498db',       # Blue for positive values
    edgecolor='#2980b9'    # Darker blue for edges
)

# Color negative bars in red
for i, bar in enumerate(bars.patches):
    if bar.get_height() < 0:
        bar.set_color('#e74c3c')    # Red for negative values
        bar.set_edgecolor('#c0392b') # Darker red for edges

# Enhance the visual style
plt.title('Maize Inflation in Kenya (2008–2025)', 
          fontsize=16, 
          pad=20,
          weight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Inflation', fontsize=12)

# Customize grid
plt.grid(True, linestyle='--', alpha=0.3, axis='y')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add value labels on top/bottom of each bar
for i, row in enumerate(filtered_data.itertuples()):
    if pd.notna(row.inflation_maize):
        value = row.inflation_maize
        va = 'bottom' if value >= 0 else 'top'  # Adjust label position based on value
        plt.text(i, value, 
                f'{value:.1f}', 
                ha='center', 
                va=va,
                fontsize=10)

# Adjust layout
plt.tight_layout()

# Set y-axis limits with padding for both positive and negative values
valid_values = filtered_data['inflation_maize'].dropna()
if not valid_values.empty:
    max_val = max(valid_values)
    min_val = min(valid_values)
    plt.ylim(min_val * 1.1, max_val * 1.1)  # Add 10% padding on both ends

# Add horizontal line at y=0
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)

plt.show()

## Potatoes Inflation over the years (2008 - 2025)

In [ ]:
# Filter out 2007 and enhance the visualization
sns.set_theme(style='whitegrid')

plt.figure(figsize=(12, 6))

# Filter data to remove 2007 and get non-NaN values
filtered_data = yearly_trends[
    (yearly_trends['year'] > 2007) & 
    (yearly_trends['inflation_potatoes'].notna())
]

# Create the bar plot
bars = sns.barplot(
    data=filtered_data,
    x='year',
    y='inflation_potatoes',
    color='#3498db',       # Blue for positive values
    edgecolor='#2980b9'    # Darker blue for edges
)

# Color negative bars in red
for i, bar in enumerate(bars.patches):
    if bar.get_height() < 0:
        bar.set_color('#e74c3c')    # Red for negative values
        bar.set_edgecolor('#c0392b') # Darker red for edges

# Enhance the visual style
plt.title('Potatoes Inflation in Kenya (2008–2025)', 
          fontsize=16, 
          pad=20,
          weight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Inflation', fontsize=12)

# Customize grid
plt.grid(True, linestyle='--', alpha=0.3, axis='y')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add value labels on top/bottom of each bar
for i, row in enumerate(filtered_data.itertuples()):
    if pd.notna(row.inflation_potatoes):
        value = row.inflation_potatoes
        va = 'bottom' if value >= 0 else 'top'  # Adjust label position based on value
        plt.text(i, value, 
                f'{value:.1f}', 
                ha='center', 
                va=va,
                fontsize=10)

# Adjust layout
plt.tight_layout()

# Set y-axis limits with padding for both positive and negative values
valid_values = filtered_data['inflation_potatoes'].dropna()
if not valid_values.empty:
    max_val = max(valid_values)
    min_val = min(valid_values)
    plt.ylim(min_val * 1.1, max_val * 1.1)  # Add 10% padding on both ends

# Add horizontal line at y=0
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)

plt.show()